# Testinterview

Zhengkun Li    

TU Darmstadt    

Kontakt:15226748638   

linkedin:linkedin.com/in/zhengkun-li-ab7b4019b

In [175]:
import csv
import os
import numpy as np
import pandas as pd
from collections import Counter

## Load the data

In [176]:
df = pd.read_csv('Customerattributiondata.csv',sep=';')

## Date time

In [177]:
df['TIMESTAMP_TOUCHPOINT'] = pd.to_datetime(df['TIMESTAMP_TOUCHPOINT'])
df['TIMESTAMP_TOUCHPOINT'].head()

0   2020-05-01 19:54:00
1   2019-12-29 12:56:00
2   2020-06-01 15:24:00
3   2019-12-26 11:18:00
4   2020-05-01 15:17:00
Name: TIMESTAMP_TOUCHPOINT, dtype: datetime64[ns]

## Number of the marketingchannel

In [178]:
mc = df['MARKETINGCHANNEL']
mc.drop_duplicates(inplace = True)
print(mc)

0                      Direct_NON-BRAND
2                              Referral
3                             SEO_BRAND
5                         SEO_NON-BRAND
64                             Adwords/
229                  Referral_NON-BRAND
4637     Social Media organic_NON-BRAND
13149                               NaN
Name: MARKETINGCHANNEL, dtype: object


## Delete the NaN value(df1)

In [179]:
df1 = df.dropna(axis=0,subset=['TIMESTAMP_TOUCHPOINT'],inplace=False)
df1.tail()

,CUSTOMERID,SESSIONID,TIMESTAMP_TOUCHPOINT,MARKETINGCHANNEL,REVENUE
13144,ffd92a1bee90652c,b9f20f189ef805e4eeca8eb0736be5f7fa515dff,2019-11-24 13:00:00,Referral,NaN
13145,ffd92a1bee90652c,d1664b86878bb91531cb38c62298fd5aa5f44581,2019-11-23 22:26:00,SEO_BRAND,NaN
13146,ffe3d2b4ffa544a0,e46d277a01168a2e18c7ba4cd8dba014e6be8360,2019-12-27 20:09:00,Direct_NON-BRAND,NaN
13147,ffe8ca2b9414bfe6,1294b5d8763cc4932c99938cd1d6cd17f3fbfa6e,2019-06-12 19:29:00,SEO_BRAND,1000.0
13148,fff79a660283ed02,e9cf96d25a8cd71a6e7ad0929547c529224405df,2019-12-28 22:57:00,Referral,NaN


## Indexing by the date&time

In [180]:
df_datesIndex = df1.set_index('TIMESTAMP_TOUCHPOINT')  # date & time indexing
df_Ascending = df1.set_index('TIMESTAMP_TOUCHPOINT').sort_index(ascending=True) # ascending date
df_Ascending.head()

,CUSTOMERID,SESSIONID,MARKETINGCHANNEL,REVENUE
TIMESTAMP_TOUCHPOINT,,,,
2019-01-12 00:02:00,ad24944249251df4,6a7b4ba75800e88428b5a3fde203d94253581bf4,SEO_NON-BRAND,1000.0
2019-01-12 00:07:00,0f01f8a3263b160c,bf0e4ec1cfaeaafac89c73e203a94ca85d91e75b,SEO_NON-BRAND,NaN
2019-01-12 00:08:00,4b04c2a7424e4eb8,f4ff0edd3f9dfb3b7507ba8ae561e8bbcac89b80,SEO_NON-BRAND,NaN
2019-01-12 00:14:00,989bef2a5c68d5ef,c47877c91a7f0011e6a006c323dce05b0860ac2d,Direct_NON-BRAND,NaN
2019-01-12 00:15:00,314a78e3b8529e29,052a3106c1b734fa99843ca91e02e97038b7b0c5,SEO_NON-BRAND,NaN


## Dict of customers

In [181]:
customer = df_Ascending['CUSTOMERID'].unique()
customer_set = set(customer)
customer = dict.fromkeys(customer, '')
customer_list = list(customer_set)

## Computing the path to conversion

In [182]:
for i in range(0, 13148):
    if df_Ascending.iat[i, 0] in customer_set:
        customer[df_Ascending.iat[i, 0]] = customer[df_Ascending.iat[i, 0]]+' '+str(df_Ascending.iat[i, 2])
        if df_Ascending.iat[i, 3] == 1000:
            customer[df_Ascending.iat[i, 0]] = customer[df_Ascending.iat[i, 0]]+' '+':True|'

## Load the path in dict into list and split the path

In [183]:
for i , key in enumerate(customer):
    customer_list[i] = customer[key]

In [185]:
#define the path list
path_list = []
for paths in customer_list:# each customer has several paths
    for path in paths.split('|'):# spplit the path with conversion
        path_list.append(path)
path_list_final = path_list.copy()
# path_list

In [188]:
for i , path in enumerate(path_list):
    path_list_final[i] = list(set(path.split(' ')))# keep the original order and delete the duplicate
    path_list_final[i].sort(key = path.index)
    if path_list_final[i][-1] != ':True':
        path_list_final[i].append(':False')

In [189]:
for path in path_list_final:
    path.remove('')
for i,path in enumerate(path_list_final):
    a = path[0]
    for item in path[1:]:
        a=a+' '+str(item)
    path_list_final[i] = a
result = dict(Counter(path_list_final))
result.pop(':False')

848

## Types of combinations

In [190]:
comb = []
for key in result:
    comb.append(key.split(':')[0])
comb_dict = dict.fromkeys(set(comb), 0)


## Path combinations and their corresponnding statical probability

In [192]:
for keys in list(comb_dict.keys()):
    if (keys+':True') in result:
        num_true = result[keys+':True']
        if (keys+':False') in result:
            num_false = result[keys+':False']
        else:
            num_true = 0
        comb_dict[keys] = num_true/(num_true+num_false)
    else:
        comb_dict[keys] = 0
# comb_dict

### List of node

In [193]:
one_node = list(mc)
one_node.pop()
mc_ohne_nan = dict.fromkeys(set(one_node), 0)
for keys in list(comb_dict.keys()):
    if len(keys.split(' ')) == 2:
        mc_ohne_nan[keys.split(' ')[0]] =  comb_dict[keys] 
df_one = pd.DataFrame(mc_ohne_nan,index=[0])
list_node = list(mc_ohne_nan.keys())

In [194]:
two_node = {}
for keys in list(comb_dict.keys()):
    if len(keys.split(' ')) == 3:
        two_node[keys] =  comb_dict[keys] 
df_two = pd.DataFrame(two_node,index=[0])
df_two

,SEO_BRAND SEO_NON-BRAND,SEO_NON-BRAND Referral,SEO_BRAND Adwords/,Referral Adwords/,Direct_NON-BRAND Adwords/,SEO_BRAND Direct_NON-BRAND,Direct_NON-BRAND Referral,Adwords/ Direct_NON-BRAND,Referral SEO_NON-BRAND,Referral Direct_NON-BRAND,SEO_NON-BRAND SEO_BRAND,SEO_BRAND Referral,Direct_NON-BRAND SEO_NON-BRAND,Referral SEO_BRAND,SEO_NON-BRAND Direct_NON-BRAND,Adwords/ SEO_BRAND,Direct_NON-BRAND SEO_BRAND
0,0.066667,0.166667,0,0,0,0.193277,0.181818,0.0,0.130435,0.184211,0.222222,0.162791,0.233333,0.12,0.157895,0.5,0.127451


## Calculating the MCF Data-Driven Attribution methodology

### Reference:
https://support.google.com/analytics/answer/3191594?hl=en

https://en.wikipedia.org/wiki/Shapley_value

The following code will be written strictly follwed by the equation below:


$\phi_i(\upsilon)=\sum_ {S\subseteq N \setminus{{i}}}^{}{\frac{\mid S\mid!(n - \mid S\mid -1)!}{n!}}(\upsilon(S\cup{i})-\upsilon(S))$

#### Generate the dict based on the length of route

In [195]:
route_length = [{} for i in range(7)]
for keys in list(comb_dict.keys()):
    node_num = len(keys.split(' '))-2
    route_length[node_num][keys] = comb_dict[keys] 
# route_length

#### calculate the marginal contribution

In [196]:
data ={'sum':np.zeros(7),'count':np.zeros(7),}# count and sum of  the marginal contribution of each marketing channel.
mc_contribution = pd.DataFrame(data,index=list_node)
for i in range(1,4):
    for route_with_specific_length in route_length[i]:
        route = route_with_specific_length.split(' ')
        for j,node_route in enumerate(route):
            for node_in_list in set(list_node):
                if node_route == node_in_list:  
                    route.pop(j)
                    for route_compared in route_length[i-1]:
                        if ' '.join(route) == route_compared:
#                             print(node_in_list+' = '+route_with_specific_length+ ' - '+route_compared)# how did each marginal contribution is calculated
#                             print(str(comb_dict[route_with_specific_length])+ ' - '+str(comb_dict[route_compared]))
                            mc_contribution.loc[node_in_list]['count'] = mc_contribution.loc[node_in_list]['count'] +1
                            mc_contribution.loc[node_in_list]['sum'] = mc_contribution.loc[node_in_list]['sum'] + route_length[i][route_with_specific_length] - route_length[i-1][route_compared]
#                             print(mc_contribution[node_in_lst]['count'])
for route_with_specific_length in route_length[0]: # add the route length = 1 contribution
    a = route_with_specific_length.split(' ')[0]
    mc_contribution.loc[a]['count'] = mc_contribution.loc[a]['count'] +1
    mc_contribution.loc[a]['sum'] = mc_contribution.loc[a]['sum'] + route_length[0][route_with_specific_length] 
# mc_contribution

In [197]:
data ={'Contribution':np.zeros(7)}# final
mc_contribution_final = pd.DataFrame(data,index=list_node)
for node in list_node:
    if mc_contribution.loc[node]['count'] != 0:
        mc_contribution_final.loc[node] = mc_contribution.loc[node]['sum'] / mc_contribution.loc[node]['count']
mc_contribution_final

,Contribution
Direct_NON-BRAND,-0.045574
Social Media organic_NON-BRAND,0.000000
Referral,0.018937
Adwords/,0.024678
Referral_NON-BRAND,0.142857
SEO_BRAND,-0.009738
SEO_NON-BRAND,-0.061305


This table means the effect of each marketing channel on the final conversion rate. For example the 'Referral' on the Marketing route will increase the conversion rate for 1.8937%.

# Summary

The final contribution of each marketing channel I computed using Shapley value is partially correct since a marketing channel can't have a negative effect on the conversion rate. The reason for this strange problem is probably that to calculate the Shapley value I have to calculate the average value of the marginal contribution so I didn't add the weight of the conversion frequency. Another probable reason for this problem is that the number of paths is missing. We have a total of 7 different marketing channels, which means, to calculate the Shapley value, I have to calculate the 7! different permutations of different marketing channels, but from the raw data I receive, there are only 54 different permutations. So if there are more data, this model may have a better outcome.